In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
import json
from request_function import request_url

In [2]:
import os

In [3]:
help(request_url)

Help on function request_url in module request_function:

request_url(*param, req='search', form='json', api_url='http://api.duma.gov.ru/api/', token='26d686e46758658adb812b6924d708e99c8a4706', app_token='app61e9cd3433b223ef90ecdcf059b6ccb48338743c', mode=None)
    Function for API GET-request
    
    Parameters
    ----------
    :param param: 
    List of request parameters objects
        list, tuple
    
    :param req: 
    Type of request
        string, default 'search'
    
    :param form:
    Type of data in response
        string, default 'json'
    
    :param api_url, token, app_token:
    Components of requested url
        default 'http://api.duma.gov.ru/api/', API_KEY, APP_KEY
    
    :param mode:
    If 'print' - is printed text form of response
        string, default None



In [4]:
# список всех депутатов и сенаторов
dep_and_sen_all = request_url('', req='deputies')

200 Success!


In [13]:
dep_and_sen_all_extracted = pd.io.json.json_normalize(dep_and_sen_all.json())

In [14]:
dep_and_sen_all_extracted

,id,name,position,isCurrent,factions
0,99111772,Абалаков Александр Николаевич,Депутат ГД,False,"[{'id': '72100004', 'name': 'Фракция Политичес..."
1,99112284,Абасов Мамед Магарамович,Депутат ГД,False,"[{'id': '72100024', 'name': 'Фракция Всероссий..."
2,99100491,Абдулатипов Рамазан Гаджимурадович,Депутат ГД,False,"[{'id': '72100020', 'name': 'Депутатская групп..."
3,99100001,Абдулбасиров Магомедтагир Меджидович,Депутат ГД,False,"[{'id': '72100001', 'name': 'Фракция Аграрной ..."
4,99100002,Абельцев Сергей Николаевич,Депутат ГД,False,"[{'id': '72100005', 'name': 'Фракция Политичес..."
5,99100635,Абраменков Дмитрий Николаевич,Депутат ГД,False,"[{'id': '72100004', 'name': 'Фракция Политичес..."
6,99110156,Абрамов Виктор Семенович,Член СФ,True,"[{'id': '72100011', 'name': 'Депутаты, не вход..."
7,99111774,Абрамов Иван Николаевич,Член СФ,True,"[{'id': '72100005', 'name': 'Фракция Политичес..."
8,99100728,Абрамович Роман Аркадиевич,Депутат ГД,False,"[{'id': '72100011', 'name': 'Депутаты, не вход..."
9,99111998,Аброськин Николай Павлович,Депутат ГД,False,"[{'id': '72100024', 'name': 'Фракция Всероссий..."


## JSON
Результат представляет собой массив объектов, каждый из которых представляет депутата ГД или члена СФ:

**id** — идентификатор депутата ГД или члена СФ

**name** — ФИО

**position** — должность («Депутат ГД» или «Член СФ»)

**isCurrent** — депутат ГД или член СФ в данной должности на данный момент

**factions** — фракция, список, в котором находится словарь либо NaN

In [22]:
dep_and_sen_all_extracted['factions'][4]

[{'id': '72100005',
  'name': 'Фракция Политической партии ЛДПР - Либерально-демократической партии России',
  'startDate': '1994-01-12',
  'endDate': '1996-01-15'},
 {'id': '72100005',
  'name': 'Фракция Политической партии ЛДПР - Либерально-демократической партии России',
  'startDate': '1996-01-16',
  'endDate': '2000-01-17'},
 {'id': '72100011',
  'name': 'Депутаты, не входящие во фракции',
  'startDate': '2003-12-07',
  'endDate': '2003-12-28'},
 {'id': '72100005',
  'name': 'Фракция Политической партии ЛДПР - Либерально-демократической партии России',
  'startDate': '2003-12-29',
  'endDate': '2008-01-07'},
 {'id': '72100005',
  'name': 'Фракция Политической партии ЛДПР - Либерально-демократической партии России',
  'startDate': '2008-03-26',
  'endDate': '2011-12-20'}]

In [17]:
for i, j in enumerate(dep_and_sen_all_extracted['factions']):
    print(i, j)
    if pd.isna(j):
        dep_and_sen_all_extracted['factions'][i] = [{'id': None, 'name': None, 'startDate': None, 'endDate': None}]

0 [{'id': '72100004', 'name': 'Фракция Политической партии "КОММУНИСТИЧЕСКАЯ ПАРТИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ"', 'startDate': '2011-12-04', 'endDate': '2016-10-04'}]
1 [{'id': '72100024', 'name': 'Фракция Всероссийской политической партии "ЕДИНАЯ РОССИЯ"', 'startDate': '2013-03-06', 'endDate': '2016-10-04'}]
2 [{'id': '72100020', 'name': 'Депутатская группа "Российские регионы"', 'startDate': '1996-01-16', 'endDate': '1997-11-12'}, {'id': '72100024', 'name': 'Фракция Всероссийской политической партии "ЕДИНАЯ РОССИЯ"', 'startDate': '2011-12-04', 'endDate': '2013-01-27'}]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [9]:
def nan_to_fillers(data):
    print(str(data))
    if data:
        return data
    elif not data:
        return [{'id': None,
                 'name': None,
                 'startDate': None,
                 'endDate': None}]

In [10]:
def list_of_faction_to_columns(data):
    print(data[0])
    return pd.io.json.json_normalize(data[0])

In [11]:
dep_and_sen_all_extracted['factions'] = dep_and_sen_all_extracted['factions'].apply(nan_to_fillers)

[{'id': '72100004', 'name': 'Фракция Политической партии "КОММУНИСТИЧЕСКАЯ ПАРТИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ"', 'startDate': '2011-12-04', 'endDate': '2016-10-04'}]
[{'id': '72100024', 'name': 'Фракция Всероссийской политической партии "ЕДИНАЯ РОССИЯ"', 'startDate': '2013-03-06', 'endDate': '2016-10-04'}]
[{'id': '72100020', 'name': 'Депутатская группа "Российские регионы"', 'startDate': '1996-01-16', 'endDate': '1997-11-12'}, {'id': '72100024', 'name': 'Фракция Всероссийской политической партии "ЕДИНАЯ РОССИЯ"', 'startDate': '2011-12-04', 'endDate': '2013-01-27'}]
[{'id': '72100001', 'name': 'Фракция Аграрной партии России', 'startDate': '1994-01-12', 'endDate': '1994-07-23'}]
[{'id': '72100005', 'name': 'Фракция Политической партии ЛДПР - Либерально-демократической партии России', 'startDate': '1994-01-12', 'endDate': '1996-01-15'}, {'id': '72100005', 'name': 'Фракция Политической партии ЛДПР - Либерально-демократической партии России', 'startDate': '1996-01-16', 'endDate': '2000-01-17'}, 

[{'id': '72100010', 'name': 'Фракция "Отечество - Единая Россия"', 'startDate': '2000-01-18', 'endDate': '2003-12-28'}]
nan
nan
[{'id': '72100009', 'name': 'Фракция "Единство - Единая Россия"', 'startDate': '2000-01-18', 'endDate': '2001-04-24'}, {'id': '72100011', 'name': 'Депутаты, не входящие во фракции', 'startDate': '2001-04-25', 'endDate': '2001-04-27'}, {'id': '72100015', 'name': 'Фракция "Союз Правых Сил"', 'startDate': '2001-04-28', 'endDate': '2003-12-28'}]
nan
[{'id': '72100011', 'name': 'Депутаты, не входящие во фракции', 'startDate': '2003-12-07', 'endDate': '2003-12-28'}, {'id': '72100024', 'name': 'Фракция Всероссийской политической партии "ЕДИНАЯ РОССИЯ"', 'startDate': '2003-12-29', 'endDate': '2008-01-07'}, {'id': '72100011', 'name': 'Депутаты, не входящие во фракции', 'startDate': '2007-12-02', 'endDate': '2007-12-23'}, {'id': '72100024', 'name': 'Фракция Всероссийской политической партии "ЕДИНАЯ РОССИЯ"', 'startDate': '2007-12-24', 'endDate': '2011-09-04'}]
[{'id': '

In [12]:
list(dep_and_sen_all_extracted['factions'])

[[{'id': '72100004',
   'name': 'Фракция Политической партии "КОММУНИСТИЧЕСКАЯ ПАРТИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ"',
   'startDate': '2011-12-04',
   'endDate': '2016-10-04'}],
 [{'id': '72100024',
   'name': 'Фракция Всероссийской политической партии "ЕДИНАЯ РОССИЯ"',
   'startDate': '2013-03-06',
   'endDate': '2016-10-04'}],
 [{'id': '72100020',
   'name': 'Депутатская группа "Российские регионы"',
   'startDate': '1996-01-16',
   'endDate': '1997-11-12'},
  {'id': '72100024',
   'name': 'Фракция Всероссийской политической партии "ЕДИНАЯ РОССИЯ"',
   'startDate': '2011-12-04',
   'endDate': '2013-01-27'}],
 [{'id': '72100001',
   'name': 'Фракция Аграрной партии России',
   'startDate': '1994-01-12',
   'endDate': '1994-07-23'}],
 [{'id': '72100005',
   'name': 'Фракция Политической партии ЛДПР - Либерально-демократической партии России',
   'startDate': '1994-01-12',
   'endDate': '1996-01-15'},
  {'id': '72100005',
   'name': 'Фракция Политической партии ЛДПР - Либерально-демократической

In [ ]:
dep_and_sen_all_extracted['factions'] = dep_and_sen_all_extracted['factions'].apply(list_of_faction_to_columns)

In [ ]:
dep_and_sen_all_extracted.to_csv(os.path.realpath('input/dep_and_sen_all_extracted.csv'))